In [ ]:
versioninfo()

In [ ]:
using DatasetManager, LabDataSources, ParkinsonsDualTaskCoordination, C3D, DSP, DataFrames,
    PlotlyJS, HypothesisTests, Biomechanics, Statistics, Printf, PrettyTables,
    CategoricalArrays, CSV

In [ ]:
rootdir = joinpath(pkgdir(ParkinsonsDualTaskCoordination), "data")

rawpath = joinpath(rootdir, "raw")
genpath = joinpath(rootdir, "generated")

subsets = [
    DataSubset("c3d", Source{C3DFile}, joinpath(rawpath, "c3d"), "S*.c3d"),
    DataSubset("ik", OSimMotion, genpath, "Subject */ik/*.mot.gz"),
    DataSubset("events", V3DEventsSource, genpath, "Subject */events/*.tsv"),
    DataSubset("dflow", Source{RawDFlowPD}, joinpath(rawpath, "dflow"), "S*.txt"),
]

labels = Dict(
    :subject => r"(?<=S)\d+B?",
    :task => r"single|dual"
)
conds = TrialConditions((:subject,:task), labels)

trials = findtrials(subsets, conds)

modelsubset = DataSubset("model", Source{OSimModel}, joinpath(rawpath, "models"), "S*.osim"; dependent=true)
conds = TrialConditions((:model,), Dict(:model => (r".osim$" => "model" => "model")); subject_fmt=r"(?<=S)(?<subject>\d+B?)", required=(:subject,))
findtrials!(trials, [modelsubset], conds)

setfield!.(trials[findall(==("15B")∘subject, trials)], :subject, "15")

demog = CSV.read("../data/demographics.csv", DataFrame)
moreaffected_side = Dict(string.(demog[!,"Participant ID"]) .=> demog[!,"More affected Side"])
addcondition!.(trials, :ma_side => t -> moreaffected_side[subject(t)])

summarize(trials; ignoreconditions=[:ma_side])

## Data exploration

In [ ]:
# finish = conditions(trl)[:kind] == "$1" ? 80. : nothing
# events = readsegment(Segment(trl, "events"; start=25.0, finish))
# rfcidx = toindices(events["RFC"], 100)

# seg = Segment(trl, "ik"; start=25.0, finish)
# jnts = readsegment(seg; series=["hip_flexion_r", "hip_flexion_l",
#     "knee_angle_r", "knee_angle_l", "arm_flex_r", "arm_flex_l", "elbow_flex_r",
#     "elbow_flex_l"]);

In [ ]:
# function normalizedeg(signal, events)
#     mi, ma = intervalextrema(signal, events)
#     mi_μ, ma_μ = circmeand(mi), circmeand(ma)
#     signalcent = signal .- Ref(mi_μ + (ma_μ - mi_μ)/2)

#     signalcent ./= (ma_μ - mi_μ)
    
#     return signalcent
# end

In [ ]:
# let trial = first(filter(==("07")∘subject, trials))
#     finish = conditions(trial)[:task] == "dual" ? 80. : nothing
#     events = readsegment(Segment(trial, "events"; start=25.0, finish))
#     rfcidx = toindices(events["RFC"], 100)

#     seg = Segment(trial, "ik"; start=25.0, finish)
#     jnts = readsegment(seg; series=["hip_flexion_r", "hip_flexion_l",
#         "knee_angle_r", "knee_angle_l", "arm_flex_r", "arm_flex_l", "elbow_flex_r",
#         "elbow_flex_l"]);
#     lhip_pgm = mt_pgram(normalizedeg(jnts.hip_flexion_l, rfcidx); fs=100)
#     rhip_pgm = mt_pgram(normalizedeg(jnts.hip_flexion_r, rfcidx); fs=100)
#     lhip = plot(scatter(;x=freq(lhip_pgm), y=power(lhip_pgm), name="Left hip"),
#         Layout(;title="Left hip", xaxis_range=[0,2.5]))
#     rhip = plot(scatter(;x=freq(rhip_pgm), y=power(rhip_pgm), name="Right hip"),
#         Layout(;title="Right hip", xaxis_range=[0,2.5]))

#     lsho_pgm = mt_pgram(normalizedeg(jnts.arm_flex_l, rfcidx); fs=100)
#     rsho_pgm = mt_pgram(normalizedeg(jnts.arm_flex_r, rfcidx); fs=100)
#     lsho = plot(scatter(;x=freq(lsho_pgm), y=power(lsho_pgm), name="Left shoulder",),
#         Layout(;title="Left shoulder", xaxis_range=[0,2.5]))
#     rsho = plot(scatter(;x=freq(rsho_pgm), y=power(rsho_pgm), name="Right Shoulder",),
#         Layout(;title="Right Shoulder", xaxis_range=[0,2.5]))

#     cross_spectra = mt_cross_power_spectra(hcat(normalizedeg(jnts.hip_flexion_l, rfcidx), 
#         normalizedeg(jnts.hip_flexion_r, rfcidx),
#         normalizedeg(jnts.arm_flex_l, rfcidx),
#         normalizedeg(jnts.arm_flex_r, rfcidx))'; fs=100, onesided=true)

#     hip_inter_p = plot(scatter(;x=freq(cross_spectra), y=abs.(power(cross_spectra)[1,2,:]),
#         name="Hip interlimb"), Layout(;xaxis_range=[0,2.5]))
#     sho_inter_p = plot(scatter(;x=freq(cross_spectra), y=abs.(power(cross_spectra)[3,4,:]),
#         name="Shoulder interlimb"), Layout(;xaxis_range=[0,2.5]))
#     lipsi_p = plot(scatter(;x=freq(cross_spectra), y=abs.(power(cross_spectra)[1,3,:]),
#         name="Left ipsilateral interlimb"), Layout(;xaxis_range=[0,2.5]))
#     ripsi_p = plot(scatter(;x=freq(cross_spectra), y=abs.(power(cross_spectra)[2,4,:]),
#         name="Right ipsilateral interlimb"), Layout(;xaxis_range=[0,2.5]))

#     p = [ lsho sho_inter_p rsho
#       lipsi_p plot() ripsi_p
#       lhip hip_inter_p rhip ]
    
#     relayout!(p.plot, Layout(;height=800, width=1000))
#     p
# end

In [ ]:
# let trial = trial = first(filter(==("07")∘subject, trials)), jntname = "arm_flex_l"
#     finish = conditions(trial)[:task] == "dual" ? 80. : nothing
#     events = readsegment(Segment(trial, "events"; start=25.0, finish))
#     rfcidx = toindices(events["RFC"], 100)

#     seg = Segment(trial, "ik"; start=25.0, finish)
#     jnts = readsegment(seg; series=["hip_flexion_r", "hip_flexion_l",
#         "knee_angle_r", "knee_angle_l", "arm_flex_r", "arm_flex_l", "elbow_flex_r",
#         "elbow_flex_l"]);
#     # nstr = timenormalize(ParkinsonsDualTaskCoordination.centerd(jnts[!, jntname], rfcidx), rfcidx)
#     nstr = timenormalize(jnts[!, jntname], rfcidx)
#     rnstr = reshape(nstr, 100, :)
#     mins, mnidxs = findmin(rnstr, dims=1)
#     maxs, mxidxs = findmax(rnstr, dims=1)

# #     NW = [ mean(nstr[i:100:end]) for i in 1:100 ]
#     NW = vec(mean(rnstr, dims=2))

#     ss = [ scatter(;x=0:99,
#             y=rnstr[:,i],
#             line_color="rgba(89,105,112,0.50)",
#             showlegend=false,
#             name="Stride $i") for i in axes(rnstr,2)]
    
#     minmaxtrace = [
#         scatter(;x=vec(LinearIndices(rnstr)[mnidxs]) .% 100, y=vec(mins), mode="markers", name="Cycle mininum"),
#         scatter(;x=vec(LinearIndices(rnstr)[mxidxs]) .% 100, y=vec(maxs), mode="markers", name="Cycle maximum"),
#     ]

#     lyt = Layout(;
#         title="Ensemble of \"$jntname\"",
#         yaxis_hoverformat=".2r",
#         xaxis_title="Stride (%)",
#         yaxis_title="Joint angle (deg)"
#         )

#     plot([ss; minmaxtrace; scatter(;x=0:99, y=NW, line_color="rgb(216,0,50)", name="Ensemble average")],lyt)
# end

In [ ]:
# let trial = first(filter(==("07")∘subject, trials)), nameA = "arm_flex_l", nameB = "arm_flex_r"
#     finish = conditions(trial)[:task] == "dual" ? 80. : nothing
#     events = readsegment(Segment(trial, "events"; start=25.0, finish))
#     rfcidx = toindices(events["RFC"], 100)

#     seg = Segment(trial, "ik"; start=25.0, finish)
#     jnts = readsegment(seg; series=["hip_flexion_r", "hip_flexion_l",
#         "knee_angle_r", "knee_angle_l", "arm_flex_r", "arm_flex_l", "elbow_flex_r",
#         "elbow_flex_l"]);
#     sigA = jnts[!, nameA]
#     sigB = jnts[!, nameB]
#     events = rfcidx
#     θA = continuousphase(sigA, events; centerfun=ParkinsonsDualTaskCoordination.centerd)
#     θB = continuousphase(sigB, events; centerfun=ParkinsonsDualTaskCoordination.centerd)
    
#     crp = unwrap(θA; range=2pi) - unwrap(θB; range=2pi)
#     crp_norm = rad2deg.(timenormalize(crp, events))

#     rnstr = unwrap(reshape(crp_norm, 100, :), range=360, dims=1)

#     NW = mod.(vec(circmeand.(eachslice(rnstr, dims=1))),360)
#     meansd = vec(circstdd.(eachslice(rnstr, dims=1)))

#     ss = [ scatter(;x=0:99,
#             y=mod.(rnstr[:,i],360),
#             line_color="rgba(89,105,112,0.50)",
#             showlegend=false,
#             yaxis="y2",
#             name="Stride $i") for i in axes(rnstr,2)]
    
#     lyt = Layout(;
#         title="Ensemble",
#         yaxis_hoverformat=".2r",
#         xaxis_title="Stride (%)",
#         yaxis_title="CRP SD (deg)",
#         xaxis_position=0,
#         yaxis=attr(;
#             side="right",
#             constraintoward="bottom",
# #             domain=[0,0.3],
#         ),
#         yaxis2=attr(;
#             overlaying="y",
#             scaleanchor="y",
#             scaleratio=1/2,
#             title="Relative phase (deg)",
#         ),
#         )

#     plot([scatter(;x=0:99, y=meansd, fill="tozeroy", yaxis="y", name="Ensemble std"); ss;
#           scatter(;x=0:99, y=NW, line_color="rgb(216,0,50)", yaxis="y2", name="Ensemble average")],lyt)
# end

## Main analysis

In [ ]:
srs = analyzedataset(trials, OSimMotion) do trial
    analyze(trial; genpath)
end;

## Statistics

In [ ]:
longdf = DatasetManager.stack(srs, [:task,:ma_side])
levels!(longdf.task, ["single", "dual"])
ordered!(longdf.task, true)
sort!(longdf, [:subject,:task])

widedf = unstack(longdf)
gd = groupby(widedf, :task);

In [ ]:
CSV.write("../results/results.csv", longdf)

In [ ]:
write_results("../results/results-wide.csv", longdf, [:task])

In [ ]:
vars = resultsvariables(srs)
degreevars = vars[findall(contains(r"flex|rom(?!_sd)|phase"), vars)]
nondegreevars = setdiff(vars, degreevars);

In [ ]:
sort!(combine(gd, [:numlsteps, :numrsteps] => ((l,r) -> [(mean([l;r]), std([l;r]), minimum([l;r]))]) => [:steps_avg, :steps_std, :steps_min]), :task)

In [ ]:
combine(widedf, :gait_speed => (v -> [(mean(skipmissing(v)), std(skipmissing(v)), minimum(skipmissing(v)))]) => [:avg_gaitvelocity, :gaitvelocity_std, :min_gaitvelocity])

### Results inspection/verification

In [ ]:
# using StatsPlots, Distributions

In [ ]:
# i = 0

In [ ]:
# i += 1
# qqplot(Normal, widedf[!, nondegreevars[i]], title=nondegreevars[i])

In [ ]:
# bp = plot([
#     box(;x=widedf.task, y=widedf[!, nondegreevars[i]], name=nondegreevars[i], legendgroup=nondegreevars[i], text=widedf.subject, hoverinfo="y+text", boxpoints="suspectedoutliers") for i in axes(nondegreevars,1)
# ], Layout(;
#     boxmode="group",
# ))
# diffp = plot([
#     scatter(;x=vec(reshape([ reshape(widedf.task, (2, :)); fill(missing, (1,size(widedf, 1)÷2)) ], (:,1))),
#             y=vec(reshape([ reshape(widedf[!, nondegreevars[i]], (2, :)); fill(missing, (1,size(widedf, 1)÷2)) ], (:,1))),
#             text=vec(reshape([ reshape(widedf.subject, (2, :)); fill(missing, (1,size(widedf, 1)÷2)) ], (:,1))),
#             legendgroup=nondegreevars[i],
#             name=nondegreevars[i], hoverinfo="y+text", mode="markers+lines") for i in axes(nondegreevars,1)
# ], Layout(;
#         width=500,
#         height=400,
# ))
# [bp diffp]

In [ ]:
# degplot = PlotlyJS.plot(reduce(vcat, collect([
#     scatterpolar(;r=ones(sum(widedf.task .== "dual")), theta=widedf[widedf.task .== "dual", degreevars[i]],
#         text=widedf[widedf.task .== "dual", :subject], hoverinfo="theta+text", mode="markers", marker_color="red", legendgrouptitle=degreevars[i], legendgroup=degreevars[i], name=degreevars[i]),
#     scatterpolar(;r=ones(sum(widedf.task .== "single")), theta=widedf[widedf.task .== "single", degreevars[i]],
#         text=widedf[widedf.task .== "single", :subject], hoverinfo="theta+text", mode="markers", marker_color="blue", legendgroup=degreevars[i], showlegend=false)]
#     for i in axes(degreevars,1))),
# Layout(;
#     uirevision=rand(),
# ))

## Statistical Analysis

In [ ]:
function unzip(x::Vector{NTuple{N,T}}) where {N,T}
    out = ntuple(_ -> Vector{T}(undef, 0), N)
    unzip!(out,x)
end

function unzip!(out, x::Vector{NTuple{N,T}}) where {N,T}
    for i in eachindex(x), j in 1:N
        push!(out[j], x[i][j])
    end
    
    return out
end

function dropmissingpairs(x, y)
    a,b = unzip(filter(x -> !any(ismissing, x), zip(x, y) |> collect))
    V = Vector{nonmissingtype(eltype(a))}
    return convert(V, a), convert(V, b)
end

In [ ]:
function CohensDz(test::OneSampleTTest)
    return test.t/sqrt(test.n)
end

function HedgesGₐᵥ(mdiff, sd1, sd2)
    return mdiff/((sd1+sd2)/2)
end

In [ ]:
function ttest_table(df)
    _df = select(df, r"^(la|ma|hip|shoulder)|task|pci|asym$")
    dt = _df[_df.task .== "dual",:]
    st = _df[_df.task .== "single",:]
    varnames = names(_df, Not([:task,:ma_side]))
    ttests = [[ OneSampleTTest((-)(dropmissingpairs(dt[!, col], st[!, col])...)) for col in intersect(nondegreevars, varnames) ];
        [ OneSampleTTest(circmeand((-)(dropmissingpairs(dt[!, col], st[!, col])...)),
                circstdd((-)(dropmissingpairs(dt[!, col], st[!, col])...)), length(dropmissingpairs(dt[!, col], st[!, col])[1]), 0) for col in intersect(degreevars, varnames) ]]
    variables = [intersect(string.(nondegreevars), varnames); intersect(string.(degreevars), varnames)]
    tails = map(variables) do var
        if occursin(r"(rom|flex)$", var)
            return :left
        elseif occursin(r"pci|meansd$", var)
            return :right
        end
        return :both
    end
    
    outdf = DataFrame(
        variables = variables,
        ST = map(variables) do var
            if var ∈ degreevars
                return Printf.format(Printf.Format("%.1f ± %.1f"), circmeand(st[!,var] |> skipmissing), circstdd(st[!,var] |> skipmissing))
            else
                return Printf.format(Printf.Format("%.1f ± %.1f"), mean(st[!,var] |> skipmissing), std(st[!,var] |> skipmissing))
            end
        end,
        DT = map(variables) do var
            if var ∈ degreevars
                return Printf.format(Printf.Format("%.1f ± %.1f"), circmeand(dt[!,var] |> skipmissing), circstdd(dt[!,var] |> skipmissing))
            else
                return Printf.format(Printf.Format("%.1f ± %.1f"), mean(dt[!,var] |> skipmissing), std(dt[!,var] |> skipmissing))
            end
        end,
        meandiff = round.(getfield.(ttests, :xbar); sigdigits=3),
        low_ci = round.(first.(confint.(ttests)); digits=1),
        upper_ci = round.(last.(confint.(ttests)); digits=1),
        t = Printf.format.(Ref(Printf.Format("t(%d)=%.2f")), getfield.(ttests, :df), getfield.(ttests, :t)),
        pvalue = ((test,t) -> clamp(round(pvalue(test; tail=t); digits=3), .001, 1)).(ttests, tails),
        tails=tails,
        Gav = Printf.format.(Ref(Printf.Format("%.2f")), HedgesGₐᵥ.(getfield.(ttests, :xbar),
                [[std(dt[!,var] |> skipmissing) for var in intersect(string.(nondegreevars), varnames)]; [circstdd(dt[!,var] |> skipmissing) for var in intersect(string.(degreevars), varnames)]],
                [[std(st[!,var] |> skipmissing) for var in intersect(string.(nondegreevars), varnames)]; [circstdd(st[!,var] |> skipmissing) for var in intersect(string.(degreevars), varnames)]])),
        CohensDz = Printf.format.(Ref(Printf.Format("%.2f")), CohensDz.(ttests)),
    )

    order = ["shoulder_rom_asym", "la_shoulder_rom", "ma_shoulder_rom", "shoulder_peakflex_asym",
        "la_shoulder_peak_flex", "ma_shoulder_peak_flex", "hip_rom_asym", "la_hip_rom",
        "ma_hip_rom", "hip_peakflex_asym", "la_hip_peak_flex", "ma_hip_peak_flex", "pci", "shoulder_inter_avg_phase",
        "shoulder_inter_meansd", "hip_inter_avg_phase", "hip_inter_meansd", "la_ipsi_avg_phase",
        "ma_ipsi_avg_phase", "ipsi_meansd_asym", "la_ipsi_meansd", "ma_ipsi_meansd", "la_contra_avg_phase",
        "ma_contra_avg_phase", "la_contra_meansd", "ma_contra_meansd", "la_ulimb_intra_avg_phase",
        "ma_ulimb_intra_avg_phase", "ulimb_intra_meansd_asym", "la_ulimb_intra_meansd",
        "ma_ulimb_intra_meansd", "la_llimb_intra_avg_phase", "ma_llimb_intra_avg_phase",
        "llimb_intra_meansd_asym", "la_llimb_intra_meansd", "ma_llimb_intra_meansd"]
    sort!(outdf, :variables; by=(x-> something(findfirst(==(x), order), 100)))
    
    outdf
end

In [ ]:
tt_results = ttest_table(widedf)
pretty_table(tt_results; backend=Val(:html), highlighters=(HTMLHighlighter((d,i,j) -> (j ∈ (8,)) && (d[i,j] ≤ .05), HTMLDecoration(font_weight = "bold")),))

In [ ]:
CSV.write("../results/ttests.csv", tt_results)